|Compuesto|CID|Usos del compuesto|
|---|---|---|
|Metano ($CH_4$)|297|Es el componente principal del **gas natural**. Lo utilizamos en nuestros hogares para cocinar en estufas, calentar agua y calefacción.|
|Etanol ($CH_3CH_2OH$)|702|Comúnmente conocido como **alcohol etilico**. Lo encontramos en: bebidas alcohólicas, desinfectantes, perfumes y productos de cuidado personal|
|Ácido Acético ($CH_3COOH$)|176|Es el principal componente del **vinagre**. Lo usamos en la cocina para: aderezar ensaladas, encurtir alimentos.|


In [1]:
# molecule de ase.build permite crear moleculas. Recibe el nombre de la molecula y las dimensiones de la celda
from ase.build import molecule

# Esta importación es para especificar el tipo de datos Atoms de ase (opcional)
from ase.atoms import Atoms

# Se importa el algoritmo de optmización BFGS (Broyden-Fletcher-Goldfarb-Shanno). Se usar para minimizar la energía de la molécula.
from ase.optimize import BFGS

# Se importa el motor de cálculo GPAW, con este se va a calcular la energía y fuerzas para la molecula (DFT)
from gpaw import GPAW

# módulo de python para interactual con el sistema operativo (se usa paracrear las carpetas)
import os

In [2]:
 def create_dir(name) -> None:
    """Crea una carpeta con un nombre dado"""
    try:
        os.mkdir(name)
        print(f"directory {name} is ready")
    except OSError as error:
        print(f"Error creating directory '{name}' : {error}")

In [3]:
def create_molecule(name: str):
    result: Atoms = molecule(name, cell=[8, 8, 8])
    # Centrar la molécula en la celda para evitar que quede en un borde.
    result.center()

    chemical_formula = result.get_chemical_formula()

    # crear la carpeta
    create_dir(chemical_formula)

    # Asigna el calculador GPAW a la molecula
    result.calc = GPAW(
        # funcional de intercambio (PBE)
        xc="PBE",

        # método de cálculo basado en orbitales lineales combinados
        mode="lcao",

        # base de datos zeta polarizada
        basis="dzp",

        # archivo donde se guardn los resultados del cálculo
        txt=f"{chemical_formula}/{chemical_formula}.txt",
    )

    # retorna el objeto Atoms anteriormente configurado
    return result

In [4]:
name_compounds = ['CH4', 'CH3CH2OH', 'CH3COOH']

In [5]:
compounds = [create_molecule(i) for i in name_compounds]

Error creating directory 'CH4' : [Errno 17] File exists: 'CH4'


[archlinux:03793] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)


Error creating directory 'C2H6O' : [Errno 17] File exists: 'C2H6O'
Error creating directory 'C2H4O2' : [Errno 17] File exists: 'C2H4O2'


In [6]:
def relax_molecule(molecule: Atoms) -> None:
    chemical_formula = molecule.get_chemical_formula()
    # se asegura que la carpeta exista
    create_dir(chemical_formula)

    # Crea un objeto optimizador que minimiza la energía total
    relax = BFGS(
        # se pasa la molecula para optimizar
        molecule,
        
        # archivo de texto con el registro del proceso de relajación
        logfile=f"{chemical_formula}/{chemical_formula}_calc.txt",
        
        # archivo donde se guarda la evolución de la estructura durante la relajación
        trajectory=f"{chemical_formula}/{chemical_formula}_calc.traj",
    )

    # Ejecuta la relajación hasta que las fuerzas sobre todos los átomos sean menores a 0.05 eV/A.
    relax.run(fmax=0.05)

In [ ]:
for i in compounds:
    relax_molecule(i)

Error creating directory 'CH4' : [Errno 17] File exists: 'CH4'
Error creating directory 'C2H6O' : [Errno 17] File exists: 'C2H6O'
